In [21]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

In [22]:
class RagState(TypedDict):
    question:str
    documents: List[Document]
    answer: str
embeddings = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-base-en-v1.5")  


In [23]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama3-70b-8192")
vectorstore = Chroma(embedding_function=embeddings)

In [24]:
docs =[
    "Python is a programming language.",
    "LangGraph helps build AI workflows.",
    "RAG combines retrieval and generation.",
    "Vector databases store embeddings."

]

In [25]:
#spillter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200)
split_docs = text_splitter.create_documents(docs)
vectorstore.add_documents(split_docs)


['9e471c42-c469-4e64-ae41-5b3062055c0e',
 '491f8835-aae7-4ef1-9c01-c21ee7f636fd',
 '8adf1306-7cfa-4a7b-baaa-17854385081a',
 '0e0489c2-c0c8-4ade-aebc-d40a50bcc55a']

In [26]:
#retrieval function
def retrieve(state: RagState) -> RagState:
    """Generate  answer using retrieved docs"""
    results = vectorstore.similarity_search(state["question"], k=2)
    return {"documents": results}

In [27]:
def generate(state: RagState) -> RagState:
    """Generate answer using retrieved docs"""
    context = "\n".join([doc.page_content for doc in state["documents"]])
    prompt = f"Context: {context}\n\nQuestion: {state['question']}\nAnswer:"

    response = llm.invoke(prompt)
    return {"answer": response.content}

In [28]:
workflow = StateGraph(RagState)
workflow.add_node("retrieve", retrieve)
workflow.add_node("generate", generate)

In [29]:
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "generate")
workflow.add_edge("generate", END)

In [30]:
app = workflow.compile()

In [31]:
if __name__ == "__main__":
    question = "What is Python?"
    result = app.invoke({"question": question})
    print(f"Question: {question}")
    print(f"Answer: {result['answer']}")

Question: What is Python?
Answer: A programming language.
